In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i, c in enumerate(world_set)}

In [4]:
print(vocab)

{'m': 0, 'B': 1, 'l': 2, 'b': 3, 'a': 4, 'o': 5, 'y': 6, 'd': 7, '.': 8, 'c': 9, 'r': 10, 'w': 11, 'e': 12, 'n': 13, 'u': 14, 'i': 15, 't': 16, 'p': 17, 's': 18, "'": 19, 'g': 20, ',': 21, 'f': 22, 'k': 23, ' ': 24, 'h': 25}


In [5]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [6]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [7]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length] 
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [8]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[1, 10, 15, 9, 23, 24, 11, 4, 2, 2]
[10, 15, 9, 23, 24, 11, 4, 2, 2, 18]


In [9]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) 
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [10]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [11]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [12]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([10, 15,  9, 23, 24, 11,  4,  2,  2, 18])


In [13]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [14]:
net = Net(vocab_size, hidden_size, 2)

In [15]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [16]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [17]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ybyyyyyyyyybyyybbyyyybbyyyybyyyyyyyyyybbyyyyyybyyyyybyyyyybyybybyybbyyyyyyyyyybyyybbbbbyyyybbyyyyybbyybyyyyybyyyyybyyyyyybbyyybyyyybyybybyybbyyyyyyyyyybyyybbyyyybbyyyyyybyybyyybbyyyyybbyyyybybyyyby
ywyaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaayaaaaaaaaaayaaaaaaaaasaaaasyaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaasaaa
a    a      a         a   a        a                     a              a      a                    a      a          a                          a      a                 a                          
a                                                                                                                                                                                                    
a                                                                                                                                                                                                    
          

In [18]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

Shutter Island 소설의 서평에서 가져옴

The year is 1954. U.S. Marshal Teddy Daniels and his new partner, Chuck Aule, have come to Shutter Island, home of Ashecliffe Hospital for the Criminally Insane, to investigate the disappearance of a patient.

Multiple murderess Rachel Solando is loose somewhere on this remote and barren island, despite having been kept in a locked cell under constant surveillance.

 As a killer hurricane bears relentlessly down on them, a strange case takes on even darker, more sinister shades with hints of radical experimentation, horrifying surgeries, and lethal countermoves made in the cause of a covert shadow war.

In [19]:
#1. 생성할 문장 데이터

sentence = ("The year is 1954. U.S. Marshal Teddy Daniels and his new partner, Chuck Aule, have come to Shutter Island, home of Ashecliffe Hospital for the Criminally Insane, to investigate the disappearance of a patient."
            "Multiple murderess Rachel Solando is loose somewhere on this remote and barren island, despite having been kept in a locked cell under constant surveillance."
            "As a killer hurricane bears relentlessly down on them, a strange case takes on even darker, more sinister shades with hints of radical experimentation, horrifying surgeries, and lethal countermoves made in the cause of a covert shadow war.")

In [20]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i, c in enumerate(world_set)}

In [21]:
print(vocab)

{'v': 0, 'm': 1, 'H': 2, 'l': 3, 'U': 4, 'b': 5, 'C': 6, 'a': 7, 'D': 8, 'o': 9, 'y': 10, 'd': 11, '.': 12, 'c': 13, 'x': 14, 'R': 15, 'r': 16, 'A': 17, 'w': 18, 'e': 19, '1': 20, 'n': 21, 'u': 22, 'i': 23, 't': 24, 'p': 25, 's': 26, 'S': 27, '5': 28, 'g': 29, 'M': 30, ',': 31, '4': 32, 'h': 33, 'f': 34, 'k': 35, 'I': 36, '9': 37, ' ': 38, 'T': 39}


In [22]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 40


In [23]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [24]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length] 
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 The year i -> he year is
1 he year is -> e year is 
2 e year is  ->  year is 1
3  year is 1 -> year is 19
4 year is 19 -> ear is 195
5 ear is 195 -> ar is 1954
6 ar is 1954 -> r is 1954.
7 r is 1954. ->  is 1954. 
8  is 1954.  -> is 1954. U
9 is 1954. U -> s 1954. U.
10 s 1954. U. ->  1954. U.S
11  1954. U.S -> 1954. U.S.
12 1954. U.S. -> 954. U.S. 
13 954. U.S.  -> 54. U.S. M
14 54. U.S. M -> 4. U.S. Ma
15 4. U.S. Ma -> . U.S. Mar
16 . U.S. Mar ->  U.S. Mars
17  U.S. Mars -> U.S. Marsh
18 U.S. Marsh -> .S. Marsha
19 .S. Marsha -> S. Marshal
20 S. Marshal -> . Marshal 
21 . Marshal  ->  Marshal T
22  Marshal T -> Marshal Te
23 Marshal Te -> arshal Ted
24 arshal Ted -> rshal Tedd
25 rshal Tedd -> shal Teddy
26 shal Teddy -> hal Teddy 
27 hal Teddy  -> al Teddy D
28 al Teddy D -> l Teddy Da
29 l Teddy Da ->  Teddy Dan
30  Teddy Dan -> Teddy Dani
31 Teddy Dani -> eddy Danie
32 eddy Danie -> ddy Daniel
33 ddy Daniel -> dy Daniels
34 dy Daniels -> y Daniels 
35 y Daniels  ->  Daniels a
36

In [25]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[39, 33, 19, 38, 10, 19, 7, 16, 38, 23]
[33, 19, 38, 10, 19, 7, 16, 38, 23, 26]


In [26]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) 
Y = torch.LongTensor(y_data)

In [27]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([594, 10, 40])
레이블의 크기 : torch.Size([594, 10])


In [28]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [29]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([33, 19, 38, 10, 19,  7, 16, 38, 23, 26])


In [30]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [31]:
net = Net(vocab_size, hidden_size, 2)

In [32]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [33]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([594, 10, 40])


In [34]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

uuudDSdDdcdudDdDdDcrDSSd1DdulDdcDlDdSSdaDddddcdDdclddcDluSdDdcDudrcDldrDdlduDrDludddruulDDSDuurDDDdDdddaDddDruulDuldddSuddruddcddddrDddruudcuSrcDdDdrDddrdDdDddDDrDDScruddDDldDDlDuSrDdddSudlDdcDdlDuldSSdDDdlDddrddDdddlDdulDDddddrdduDddDcudrDducdDdAruDlDdduludDdlDdDlulDdDduunDlcdDdDlDSdlDddddaDddDddddrDlDlcdrDdDlDrDluSrDcrDcDduouulddddddDuDdDddruDdDlDdddDuDdDddrDdlDdd1SuudddDdcluudlrrDlDSdDdDDuddlDdddDdrSDddrDcuDruldDdcSddDdrDdlddDDrDdludDDrDduDdDldDuuuulduudlDdrDdDDDdculDDdddururDlrDDdDudDdSdudddDuSdDddblDdDDdrDdDruddrluaDdDdDduDdDdDdcdDdddruldddrudDDDdruDdDdlSddDdDculrddDdDlDuldSdruDDdDdulDDdDdlS
reellelllllllellellllllllelllellelllllllllllllelllllllellllellelellllllllleellllellleelllelleleelllllllllelleelllelllelllleeellelllellleeellellllllllllllllllllelllelleellllllelllellllllellllllllleelllllellleleleelllelllellllllllleelllleelllellllleelllleelllllllelllelllleleellllllelellllllllelellllllellllellllllelllellllllllleelelllllllleelllleellllellleelllllllllllllelllllllllelllleeleelllllel

In [35]:
predict_str

'eeliear is 1954. U.S. Marshal Teddy Daniels and his new partner, Chuck Aule, have come to Shutter Island, home of Ashecliffe Hospital for the criminally Insane, to investigate the disalpearance of a patient.Multiple murderess Rachel Solando is lomse somewhere on thes remote and berren island, despite haveng beer kept in a locked cell under constant survesnlance.As a keller hurrecane baars relentlemsly down on the , a strange ca e taves on even darker, more sinaster siades withehants of radical experimentatione horrifying surgeries  and lethal countermoves made in the cause of a covert shadow wark'

#한글 sentences

박경리, 작가와의 대화, 신동아 1981.5 에서 추출

In [36]:
#1. 생성할 문장 데이터

sentence = ("유교가 농민들에게도 흘러갔던 것입니다."
            "제사라든지, 의식 구조라든지, 조상 숭배라든지 이 모든 것이 농부들에게 흘러갔는데, 이것은 농부들에게 인간으로서의 존엄성을 인식시키게 되었습니다."
            "이러한 예의 범절은 우리 농민에게 미의식이 있었다는 얘기입니다.")

In [37]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i, c in enumerate(world_set)}

In [38]:
print(vocab)

{'상': 0, '게': 1, '입': 2, '것': 3, '식': 4, '을': 5, '한': 6, '얘': 7, '민': 8, '되': 9, '로': 10, '이': 11, '구': 12, '습': 13, '유': 14, '배': 15, '은': 16, '절': 17, '.': 18, '키': 19, '우': 20, '흘': 21, '부': 22, '존': 23, '기': 24, '데': 25, '에': 26, '라': 27, '가': 28, '는': 29, '었': 30, '의': 31, '던': 32, '러': 33, '농': 34, '예': 35, '제': 36, ' ': 37, '조': 38, '범': 39, '들': 40, '든': 41, '간': 42, '있': 43, '사': 44, '교': 45, '으': 46, '성': 47, '갔': 48, ',': 49, '엄': 50, '시': 51, '도': 52, '리': 53, '지': 54, '모': 55, '인': 56, '서': 57, '숭': 58, '다': 59, '미': 60, '니': 61}


In [39]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 62


In [40]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [41]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length] 
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 유교가 농민들에게도 -> 교가 농민들에게도 
1 교가 농민들에게도  -> 가 농민들에게도 흘
2 가 농민들에게도 흘 ->  농민들에게도 흘러
3  농민들에게도 흘러 -> 농민들에게도 흘러갔
4 농민들에게도 흘러갔 -> 민들에게도 흘러갔던
5 민들에게도 흘러갔던 -> 들에게도 흘러갔던 
6 들에게도 흘러갔던  -> 에게도 흘러갔던 것
7 에게도 흘러갔던 것 -> 게도 흘러갔던 것입
8 게도 흘러갔던 것입 -> 도 흘러갔던 것입니
9 도 흘러갔던 것입니 ->  흘러갔던 것입니다
10  흘러갔던 것입니다 -> 흘러갔던 것입니다.
11 흘러갔던 것입니다. -> 러갔던 것입니다.제
12 러갔던 것입니다.제 -> 갔던 것입니다.제사
13 갔던 것입니다.제사 -> 던 것입니다.제사라
14 던 것입니다.제사라 ->  것입니다.제사라든
15  것입니다.제사라든 -> 것입니다.제사라든지
16 것입니다.제사라든지 -> 입니다.제사라든지,
17 입니다.제사라든지, -> 니다.제사라든지, 
18 니다.제사라든지,  -> 다.제사라든지, 의
19 다.제사라든지, 의 -> .제사라든지, 의식
20 .제사라든지, 의식 -> 제사라든지, 의식 
21 제사라든지, 의식  -> 사라든지, 의식 구
22 사라든지, 의식 구 -> 라든지, 의식 구조
23 라든지, 의식 구조 -> 든지, 의식 구조라
24 든지, 의식 구조라 -> 지, 의식 구조라든
25 지, 의식 구조라든 -> , 의식 구조라든지
26 , 의식 구조라든지 ->  의식 구조라든지,
27  의식 구조라든지, -> 의식 구조라든지, 
28 의식 구조라든지,  -> 식 구조라든지, 조
29 식 구조라든지, 조 ->  구조라든지, 조상
30  구조라든지, 조상 -> 구조라든지, 조상 
31 구조라든지, 조상  -> 조라든지, 조상 숭
32 조라든지, 조상 숭 -> 라든지, 조상 숭배
33 라든지, 조상 숭배 -> 든지, 조상 숭배라
34 든지, 조상 숭배라 -> 지, 조상 숭배라든
35 지, 조상 숭배라든 -> , 조상 숭배라든지
36

In [42]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[14, 45, 28, 37, 34, 8, 40, 26, 1, 52]
[45, 28, 37, 34, 8, 40, 26, 1, 52, 37]


In [43]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) 
Y = torch.LongTensor(y_data)

In [44]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([127, 10, 62])
레이블의 크기 : torch.Size([127, 10])


In [45]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [46]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([45, 28, 37, 34,  8, 40, 26,  1, 52, 37])


In [47]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [48]:
net = Net(vocab_size, hidden_size, 2)

In [49]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [50]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([127, 10, 62])


In [51]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

민민 게게  , ,게,,게 , , ,,, 게게 민  ,게, ,간 민 ,,게 ,지민 게 ,게,  ,,게  , 게,,게  민 , 게게민  , 게 ,,,, 게민,, 게 ,민,민게 , 게,,  민 , 게 ,게,,게게게 , 게,,민민,,,,지게 , 게,
                                                                                                                                        
                                                                                                                                        
                                                                                                                                        
                                                                                                                                        
                                                                                                                                        
                                                                                                                                        
      든      게 것       든      게  든      게

In [52]:
predict_str

'교가 농민들에게도 흘러갔던 것입니다.제사라든지, 의식 구조라든지, 조상 숭배라든지 이 모든 것이 농부들에게 흘러갔는데, 이것은 농부들에게 인간으로서의 존엄성을 인식시키게 되었습니다.이러한 예의 범절은 우리 농민에게 미의식이 있었다는 얘기입니다.'